# Outlays for Mandatory and Related Programs 

Source: https://www.whitehouse.gov/omb/historical-tables/

Table 8.6—Outlays for Mandatory and Related Programs in Constant (FY 2009) Dollars: 1962–2023

Values in billions of dollars

In [103]:
url = "https://www.whitehouse.gov/wp-content/uploads/2018/02/hist08z6-fy2019.xlsx"

In [104]:
import pandas as pd

In [105]:
def load_data(url):
    # Parse excel file from url
    df = pd.read_excel(url, skiprows=2)
    df = df.dropna()
    df = df.replace("..........", 0)
    df.index = df['Category and Program']
    del df['Category and Program']
    
    df.columns = [i.replace(" estimate","") for i in df.columns]

    # Only include actual years, not future year estimates
    years_to_include = [str(i) for i in range(1962,2024)]
    df = df[years_to_include]
    return df

In [106]:
raw_data = load_data(url)

In [107]:
def parse_mandatory_data(df):
    
    # Define categories
    social_security = df.loc['Social Security']
    health_and_medicare = df.loc[['Total health', 'Medicare']].sum()
    income_security = df.loc['Total income security']
    veterans_benefits = df.loc['Total veterans benefits and services']
    other_mandatory = df.loc[['Education, training, employment, and social services','Total other mandatory programs']].sum()
    net_interest = df.loc['Total net interest']
    
    # Name categories
    social_security.name = 'Soc. Security'
    health_and_medicare.name = 'Health and Medicare'
    income_security.name = 'Income Security'
    veterans_benefits.name = 'Veterans Benefits'
    other_mandatory.name = 'Other'
    net_interest.name = 'Net Interest'
    
    # Sort categories in descending order (cumulative)
    unsorted_categories = [social_security, health_and_medicare, income_security,
                           veterans_benefits, other_mandatory, net_interest]
    cumulative_sums = [i.sum() for i in unsorted_categories]
    sorted_categories = [x for _, x in sorted(zip(cumulative_sums,unsorted_categories), reverse=True)]
    
    # Create dataframe
    output = pd.DataFrame(sorted_categories)
    output = output.transpose()
    return output

In [108]:
df = parse_mandatory_data(raw_data)

In [109]:
df.head()

,Soc. Security,Health and Medicare,Income Security,Net Interest,Veterans Benefits,Other
1962,78.5,0.6,48.5,38.6,23.5,2.2
1963,85.4,0.9,47.8,42.9,22.4,-6.1
1964,88.5,1.2,49.1,44.9,22.6,7.0
1965,91.7,1.6,47.8,46.2,22.3,7.4
1966,106.9,4.3,47.9,49.4,22.7,2.9


In [110]:
def build_pie_chart_data(df, frames_per_year):
    pie = df.copy()
    pie['Total'] = pie.sum(axis=1)
    
    for i in pie.columns:
        new_col_name = str(i + "_prev")
        pie[new_col_name] = pie[i].shift(1).fillna(0)
        
    pie['year'] = pie.index.astype(float)
    min_year = min(pie['year'])
    pie['start_frame'] = pie['year'].astype(float) * frames_per_year - (min_year * frames_per_year)
    pie['stop_frame'] = pie['start_frame'] + frames_per_year
    
    pie = pie.fillna(0)
    pie = pie.clip(lower=0)
    
    return pie

In [111]:
pie_df = build_pie_chart_data(df, frames_per_year = 60)

In [112]:
pie_df.to_csv("../data/mandatory_outlays.csv")

In [113]:
pie_df

,Soc. Security,Health and Medicare,Income Security,Net Interest,Veterans Benefits,Other,Total,Soc. Security_prev,Health and Medicare_prev,Income Security_prev,Net Interest_prev,Veterans Benefits_prev,Other_prev,Total_prev,year,start_frame,stop_frame
1962,78.5,0.6,48.5,38.6,23.5,2.2,191.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1962.0,0.0,60.0
1963,85.4,0.9,47.8,42.9,22.4,0.0,193.3,78.5,0.6,48.5,38.6,23.5,2.2,191.9,1963.0,60.0,120.0
1964,88.5,1.2,49.1,44.9,22.6,7.0,213.3,85.4,0.9,47.8,42.9,22.4,0.0,193.3,1964.0,120.0,180.0
1965,91.7,1.6,47.8,46.2,22.3,7.4,217.0,88.5,1.2,49.1,44.9,22.6,7.0,213.3,1965.0,180.0,240.0
1966,106.9,4.3,47.9,49.4,22.7,2.9,234.1,91.7,1.6,47.8,46.2,22.3,7.4,217.0,1966.0,240.0,300.0
1967,109.4,18.3,49.4,52.5,25.9,6.7,262.2,106.9,4.3,47.9,49.4,22.7,2.9,234.1,1967.0,300.0,360.0
1968,116.1,30.4,55.4,54.8,26.1,19.9,302.7,109.4,18.3,49.4,52.5,25.9,6.7,262.2,1968.0,360.0,420.0
1969,127.7,36.1,58.6,60.0,27.4,8.1,317.9,116.1,30.4,55.4,54.8,26.1,19.9,302.7,1969.0,420.0,480.0
1970,135.3,38.5,66.9,64.5,29.7,10.4,345.3,127.7,36.1,58.6,60.0,27.4,8.1,317.9,1970.0,480.0,540.0
1971,153.5,41.5,94.1,63.3,32.3,0.0,383.2,135.3,38.5,66.9,64.5,29.7,10.4,345.3,1971.0,540.0,600.0


In [125]:
(PI * 15**2) / (PI * 8.8**2)

2.90547520661157

In [119]:
PI * 15**2

706.85775

In [121]:
706.85775 / 201.06176

3.515625

In [81]:
value1 = 100
value2 = 400

value_min = 0
value_max = 400

diameter_min = 180
diameter_max = 600

In [82]:
true_multiple = value2 / value1
true_multiple

4.0

In [83]:
lerp1 = (value1 - value_min) / (value_max - value_min)
lerp1

0.25

In [84]:
lerp2 =(value2 - value_min) / (value_max - value_min)
lerp2

1.0

In [90]:
diameter1 = (diameter_max - diameter_min) * lerp1 + diameter_min
radius1 = diameter1 / 2.0
diameter1

285.0

In [91]:
diameter2 = (diameter_max - diameter_min) * lerp2 + diameter_min
diameter2

600.0

In [92]:
PI = 3.14159

In [99]:
area1 = PI * (diameter1/2)**2
area1

63793.9119375

In [100]:
area2 = PI * (diameter2/2)**2
area2

282743.1

In [101]:
diameter2 / diameter1

2.1052631578947367

In [102]:
area2 / area1

4.432132963988919

In [ ]:
true_diameter1